In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/02 16:25:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/02 16:25:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
print(f'The PySpark {spark.version} version is running...')

The PySpark 3.4.0 version is running...


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2024-07-23 21:45:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.46, 18.155.128.6, 18.155.128.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M   237MB/s    in 1.2s    

2024-07-23 21:45:25 (237 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [6]:
!wc -l fhvhv_tripdata_2021-01.parquet

1006794 fhvhv_tripdata_2021-01.parquet


In [5]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [6]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [7]:
from pyspark.sql import types

In [9]:
schema = types.StructType(
    [types.StructField('hvfhs_license_num', types.StringType(), True),
     types.StructField('dispatching_base_num', types.StringType(), True),
     types.StructField('originating_base_num', types.StringType(), True),
     types.StructField('request_datetime', types.TimestampType(), True),
     types.StructField('on_scene_datetime', types.TimestampType(), True),
     types.StructField('pickup_datetime', types.TimestampType(), True),
     types.StructField('dropoff_datetime', types.TimestampType(), True),
     types.StructField('PULocationID', types.IntegerType(), True),
     types.StructField('DOLocationID', types.IntegerType(), True),
     types.StructField('trip_miles', types.DoubleType(), True),
     types.StructField('trip_time', types.LongType(), True),
     types.StructField('base_passenger_fare', types.DoubleType(), True),
     types.StructField('tolls', types.DoubleType(), True),
     types.StructField('bcf', types.DoubleType(), True),
     types.StructField('sales_tax', types.DoubleType(), True),
     types.StructField('congestion_surcharge', types.DoubleType(), True),
     types.StructField('airport_fee', types.DoubleType(), True),
     types.StructField('tips', types.DoubleType(), True),
     types.StructField('driver_pay', types.DoubleType(), True),
     types.StructField('shared_request_flag', types.StringType(), True),
     types.StructField('shared_match_flag', types.StringType(), True),
     types.StructField('access_a_ride_flag', types.StringType(), True),
     types.StructField('wav_request_flag', types.StringType(), True),
     types.StructField('wav_match_flag', types.StringType(), True)
])

In [42]:
#lazy command doesn't trigger yet
#only when saved it is triggered
#expensive operation
df = df.repartition(24)

In [43]:
df.write.parquet('fhvhv/2021/01/')

In [8]:
df = spark.read.parquet('fhvhv/2021/01/')

In [9]:
# Unlike CSV files, parquet files contain the schema of the dataset, so there is no need to specify a schema. You can check the schema like this:
# (One of the reasons why parquet files are smaller than CSV files is because they store the data according to the datatypes, so integer values will take less space than long or string values.)
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [10]:
#only executes when we do .show

#similar to: SELECT * FROM df WHERE hvfhs_license_num = HV0003
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-11 18:40:22|2021-01-11 19:15:49|         262|         231|
|2021-01-05 15:13:22|2021-01-05 15:27:50|          61|         181|
|2021-01-31 18:42:09|2021-01-31 18:59:52|         232|           4|
|2021-01-27 22:24:36|2021-01-27 22:26:43|          68|          68|
|2021-01-30 08:35:46|2021-01-30 08:39:42|         256|         255|
|2021-01-16 02:25:35|2021-01-16 02:34:21|          89|          91|
|2021-01-11 11:58:23|2021-01-11 12:14:19|          97|          61|
|2021-01-03 07:44:58|2021-01-03 08:04:45|          26|         178|
|2021-01-14 18:52:00|2021-01-14 19:19:00|         181|         198|
|2021-01-08 20:35:35|2021-01-08 21:06:33|          76|          91|
|2021-01-15 13:49:48|2021-01-15 14:35:23|         246|          16|
|2021-01-27 10:37:56|2021-01-27 10:53:35|       

In [11]:
from pyspark.sql import functions as F

In [12]:
# creating new function
#Besides these built-in functions, Spark allows us to create User Defined Functions (UDFs) with custom behavior for those instances where creating SQL queries for that behaviour becomes difficult both to manage and test.
#UDFs are regular functions which are then passed as parameters to a special builder. Let's create one:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [13]:
crazy_stuff('B02884')

's/b44'

In [16]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [42]:
#adding two new columns
#if giving column name that already exists -> overwrite

df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date') \
    .show()

#    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \

+-----------+------------+
|pickup_date|dropoff_date|
+-----------+------------+
| 2021-01-11|  2021-01-11|
| 2021-01-05|  2021-01-05|
| 2021-01-02|  2021-01-02|
| 2021-01-31|  2021-01-31|
| 2021-01-05|  2021-01-05|
| 2021-01-27|  2021-01-27|
| 2021-01-18|  2021-01-18|
| 2021-01-30|  2021-01-30|
| 2021-01-16|  2021-01-16|
| 2021-01-05|  2021-01-05|
| 2021-01-11|  2021-01-11|
| 2021-01-22|  2021-01-22|
| 2021-01-03|  2021-01-03|
| 2021-01-14|  2021-01-14|
| 2021-01-08|  2021-01-08|
| 2021-01-15|  2021-01-15|
| 2021-01-27|  2021-01-27|
| 2021-01-18|  2021-01-18|
| 2021-01-11|  2021-01-11|
| 2021-01-24|  2021-01-24|
+-----------+------------+
only showing top 20 rows



In [ ]:
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

In [19]:
from pyarrow.parquet import ParquetFile
pf = ParquetFile('fhvhv_tripdata_2021-01.parquet')
#pyarrow builds tables, not dataframes
tbl_small = next(pf.iter_batches(batch_size = 1000))
#this function converts the table to a dataframe of manageable size
df_small = tbl_small.to_pandas()

In [20]:
df_small.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02682,B02682,2021-01-01 00:28:09,2021-01-01 00:31:42,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,5.26,...,1.98,2.75,NaN,0.00,14.99,N,N,,N,N
1,HV0003,B02682,B02682,2021-01-01 00:45:56,2021-01-01 00:55:19,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,3.65,...,1.63,0.00,NaN,0.00,17.06,N,N,,N,N
2,HV0003,B02764,B02764,2021-01-01 00:21:15,2021-01-01 00:22:41,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,3.51,...,1.25,2.75,NaN,0.94,12.98,N,N,,N,N
3,HV0003,B02764,B02764,2021-01-01 00:39:12,2021-01-01 00:42:37,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,0.74,...,0.70,2.75,NaN,0.00,7.41,N,N,,N,N
4,HV0003,B02764,B02764,2021-01-01 00:46:11,2021-01-01 00:47:17,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,9.20,...,2.41,2.75,NaN,0.00,22.44,N,N,,N,N


In [23]:
df_small.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
dtype: object

In [25]:
!head -n 100 fhvhv_tripdata_2021-01.parquet > head.parquet